In [ ]:
from scipy.stats import multivariate_normal
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from random import choices
from glob import glob
import xarray as xr
import datetime as dt

## This notebook explores the Gaussian Plume dispersion model along with a simple inverse example that assumes *in situ* observations at the surface.

### Author: Sean Crowell
### Date: XXX
### Purpose: SSIM-GHG 2025

### Gaussian Plume solution with constant eddy diffusivity 

In [ ]:
def gaus_plume(emis_rate=None,advect_wind=None,sig_y=None,sig_z=None,coords=None):
    x,y,z,H = coords[:]
    dq = emis_rate/(4*np.pi*advect_wind*sig_y*sig_z)
    dq *= np.exp(-y**2/sig_y**2)
    dq *= np.exp(-(z+H)**2/sig_z**2) + np.exp(-(z-H)**2/sig_z**2)
    return dq

In [ ]:
Q = 1000/60/60 #kg/s
u = 5 #m/s
k = 5 #m^2/s
H = 0
x = np.arange(0,11,0.01)
sig_y = np.sqrt(2*k/u*x)
sig_z = np.sqrt(2*k/u*x)
y = np.arange(-10,11,0.5)
z = np.arange(np.max((H-10,0)),H+11,1)
dq = np.zeros((z.shape[0],y.shape[0],x.shape[0]))
for iz,zz in enumerate(z):
    for iy,yy in enumerate(y):
        for ix,xx in enumerate(x):
            dq[iz,iy,ix] = gaus_plume(emis_rate=Q,advect_wind=u,sig_y=sig_y[ix],sig_z=sig_z[ix],coords=np.array([xx,yy,zz,H]))
plt.pcolormesh(x,y,dq[0],vmin=0.001,cmap=plt.cm.hot_r); plt.colorbar()
plt.plot(x,np.sqrt(2*k/u*x))
plt.gca().set_xlim([0,1])

In [ ]:
plt.plot(y,dq[0,:,1])
plt.plot(y,dq[0,:,5])
plt.plot(y,dq[0,:,10])

plt.figure()
plt.plot(dq[:,:,:].sum((0,1)))

In [ ]:
dq[0]